# SPIN2 Part-Level Instance Segmentation Dataset Visualization

A professional visualization tool for exploring the SPIN2 dataset with part-level annotations.

## Features
- 🎲 **Random Exploration**: View random images from the dataset
- 🔍 **Specific Image**: View specific images by ID or filename
- 🏷️ **Supercategory Filter**: Explore random images by supercategory (Biped, Quadruped, Bird, etc.)
- 🔄 **Multi-Split Support**: Works with train, test, and val splits
- 🎨 **Dual Visualization**: See both bounding boxes and segmentation masks
- 📊 **Instance Tracking**: Each instance has unique colors and labels

In [1]:
# Import required libraries
import json
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image, ImageDraw
import numpy as np
import random
import os
from matplotlib.colors import to_rgba
from pycocotools import mask as mask_utils
from collections import defaultdict

# Configure matplotlib for better visualization
plt.rcParams['figure.max_open_warning'] = 50
%matplotlib inline

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


---
## ⚙️ Configuration

**Change `DATASET_SPLIT` to view different splits:**
- `'train'` - Training set
- `'test'` - Test set
- `'val'` - Validation set

In [2]:
# ===========================
# CONFIGURATION & LOAD DATASET
# ===========================

# ⚙️ CONFIGURE YOUR SPLIT HERE
DATASET_SPLIT = 'val'  # Options: 'train', 'test', 'val'

# Path to the annotation file
json_file = f"data/annotations/spin2_{DATASET_SPLIT}_parts.json"

# Load the dataset
try:
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    print("="*70)
    print(f"📦 SPIN2 Dataset Loaded Successfully ({DATASET_SPLIT.upper()} split)")
    print("="*70)
    print(f"   📷 Images:       {len(data['images'])}")
    print(f"   📊 Annotations:  {len(data['annotations'])}")
    print(f"   🏷️  Categories:   {len(data['categories'])}")
    print(f"   🔖 Split:        {DATASET_SPLIT}")
    print("="*70)
    
    dataset_split = DATASET_SPLIT
    
except FileNotFoundError:
    print(f"❌ ERROR: File not found: {json_file}")
    print(f"\nAvailable splits:")
    for split in ['train', 'test', 'val']:
        file_path = f"data/annotations/spin2_{split}_parts.json"
        if os.path.exists(file_path):
            print(f"   ✅ {split}: {file_path}")
        else:
            print(f"   ❌ {split}: {file_path} (not found)")
    raise

📦 SPIN2 Dataset Loaded Successfully (VAL split)
   📷 Images:       519
   📊 Annotations:  2348
   🏷️  Categories:   40
   🔖 Split:        val


In [3]:
# ===========================
# BUILD CATEGORY MAPPINGS
# ===========================

# Create category mappings
categories = {cat['id']: cat['name'] for cat in data['categories']}
category_to_supercategory = {cat['name']: cat.get('supercategory', 'Unknown') for cat in data['categories']}
supercategory_to_categories = defaultdict(list)

for cat in data['categories']:
    supercategory = cat.get('supercategory', 'Unknown')
    supercategory_to_categories[supercategory].append(cat['id'])

# Get unique supercategories
supercategories = sorted(list(supercategory_to_categories.keys()))

# Build image to annotations mapping for faster lookup
image_to_annotations = defaultdict(list)
for ann in data['annotations']:
    image_to_annotations[ann['image_id']].append(ann)

# Build image to supercategories mapping
image_to_supercategories = defaultdict(set)
for ann in data['annotations']:
    cat_name = categories.get(ann['category_id'], 'Unknown')
    supercategory = category_to_supercategory.get(cat_name, 'Unknown')
    image_to_supercategories[ann['image_id']].add(supercategory)

print("\n📋 Categories and Supercategories:")
print("-" * 70)
for supercat in supercategories:
    cat_ids = supercategory_to_categories[supercat]
    cat_names = [categories[cid] for cid in cat_ids if cid in categories]
    print(f"\n🏷️  {supercat}:")
    for name in sorted(cat_names):
        print(f"    • {name}")

print("\n" + "="*70)


📋 Categories and Supercategories:
----------------------------------------------------------------------

🏷️  Aeroplane:
    • Aeroplane Body
    • Aeroplane Engine
    • Aeroplane Head
    • Aeroplane Tail
    • Aeroplane Wing

🏷️  Bicycle:
    • Bicycle Body
    • Bicycle Head
    • Bicycle Seat
    • Bicycle Tire

🏷️  Biped:
    • Biped Arm
    • Biped Body
    • Biped Head
    • Biped Leg
    • Biped Tail

🏷️  Bird:
    • Bird Body
    • Bird Foot
    • Bird Head
    • Bird Tail
    • Bird Wing

🏷️  Boat:
    • Boat Body
    • Boat Sail

🏷️  Bottle:
    • Bottle Body
    • Bottle Mouth

🏷️  Car:
    • Car Body
    • Car Side Mirror
    • Car Tire

🏷️  Fish:
    • Fish Body
    • Fish Fin
    • Fish Head
    • Fish Tail

🏷️  Quadruped:
    • Quadruped Body
    • Quadruped Foot
    • Quadruped Head
    • Quadruped Tail

🏷️  Reptile:
    • Reptile Body
    • Reptile Foot
    • Reptile Head
    • Reptile Tail

🏷️  Snake:
    • Snake Body
    • Snake Head



In [8]:
# ===========================
# HELPER FUNCTIONS
# ===========================

def create_mask_from_segmentation(segmentation, image_size):
    """
    Create a binary mask from segmentation data.
    Handles both polygon and RLE formats.
    
    Args:
        segmentation: Segmentation data (RLE dict or polygon list)
        image_size: Tuple of (width, height)
    
    Returns:
        Binary mask as numpy array or None if error
    """
    width, height = image_size
    
    if not segmentation:
        return None
    
    try:
        # Handle RLE format (dict with 'size' and 'counts')
        if isinstance(segmentation, dict) and 'counts' in segmentation:
            rle = segmentation.copy()
            if 'size' not in rle:
                rle['size'] = [height, width]
            
            # Decode RLE to binary mask
            mask = mask_utils.decode(rle)
            
            # Check if mask size matches image size
            if mask.shape[0] != height or mask.shape[1] != width:
                from PIL import Image as PILImage
                mask_img = PILImage.fromarray((mask * 255).astype(np.uint8))
                mask_img = mask_img.resize((width, height), PILImage.NEAREST)
                mask = np.array(mask_img) > 0
            
            return mask.astype(bool)
        
        # Handle polygon format (list of [x1,y1,x2,y2,...])
        elif isinstance(segmentation, list):
            if len(segmentation) == 0:
                return None
                
            if isinstance(segmentation[0], (int, float)):
                # Single polygon
                polygon = [(segmentation[i], segmentation[i+1]) for i in range(0, len(segmentation), 2)]
                mask_img = Image.new('L', (width, height), 0)
                ImageDraw.Draw(mask_img).polygon(polygon, outline=1, fill=1)
                return np.array(mask_img, dtype=bool)
            
            elif isinstance(segmentation[0], list):
                # Multiple polygons
                mask_img = Image.new('L', (width, height), 0)
                draw = ImageDraw.Draw(mask_img)
                for poly in segmentation:
                    if len(poly) >= 6:
                        polygon = [(poly[i], poly[i+1]) for i in range(0, len(poly), 2)]
                        draw.polygon(polygon, outline=1, fill=1)
                return np.array(mask_img, dtype=bool)
        
        return None
            
    except Exception as e:
        print(f"⚠️  Error creating mask: {e}")
        return None


def load_image(image_info, split=None):
    """
    Load an image from various possible paths across different splits.
    
    Args:
        image_info: Image info dictionary from COCO format
        split: Dataset split (train/test/val) - if None, uses global dataset_split
    
    Returns:
        PIL Image object or None if not found, and the path where it was found
    """
    if split is None:
        split = dataset_split
    
    # Try multiple possible paths for different splits
    possible_paths = [
        # Direct filename
        image_info['file_name'],
        
        # PartInventory_Val structure
        os.path.join(f'PartInventory_Val/images/{split}/', image_info['file_name']),
        os.path.join(f'PartInventory_Val/images/val/', image_info['file_name']),
        
        # PartImageNet structure with .JPEG extension
        os.path.join(f'data/PartImageNet/images/{split}/', image_info['file_name'] + '.JPEG'),
        os.path.join(f'data/PartImageNet/images/{split}/', image_info['file_name']),
        
        # Try all splits if current split doesn't work
        os.path.join('data/PartImageNet/images/train/', image_info['file_name'] + '.JPEG'),
        os.path.join('data/PartImageNet/images/test/', image_info['file_name'] + '.JPEG'),
        os.path.join('data/PartImageNet/images/val/', image_info['file_name'] + '.JPEG'),
        os.path.join('data/PartImageNet/images/train/', image_info['file_name']),
        os.path.join('data/PartImageNet/images/test/', image_info['file_name']),
        os.path.join('data/PartImageNet/images/val/', image_info['file_name']),
        
        # Generic data folder
        os.path.join('data', image_info['file_name']),
    ]

    for path in possible_paths:
        if os.path.exists(path):
            try:
                img = Image.open(path)
                if img.mode != 'RGB':
                    img = img.convert('RGB')
                return img, path
            except:
                continue
    
    return None, None


def get_distinct_colors(n):
    """Generate n visually distinct colors."""
    base_colors = [
        '#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', 
        '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe',
        '#008080', '#e6beff', '#9a6324', '#fffac8', '#800000',
        '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080'
    ]
    
    if n <= len(base_colors):
        return base_colors[:n]
    
    # Generate more colors if needed
    colors = base_colors.copy()
    while len(colors) < n:
        colors.append('#%06x' % random.randint(0, 0xFFFFFF))
    
    return colors[:n]


print("✅ Helper functions loaded successfully!")

✅ Helper functions loaded successfully!


In [9]:
# ===========================
# MAIN VISUALIZATION FUNCTION
# ===========================

def visualize_image(image_info, annotations=None, show_bbox=True, show_mask=True):
    """
    Visualize an image with its annotations.
    
    Args:
        image_info: Image info dictionary
        annotations: List of annotations (if None, will look up from image_id)
        show_bbox: Whether to show bounding boxes
        show_mask: Whether to show segmentation masks
    """
    # Load image
    img, img_path = load_image(image_info, split=dataset_split)
    
    if img is None:
        print(f"❌ Could not load image: {image_info['file_name']}")
        return
    
    # Get annotations for this image
    if annotations is None:
        annotations = image_to_annotations[image_info['id']]
    
    if len(annotations) == 0:
        print(f"⚠️  No annotations found for this image")
        return
    
    # Sort annotations by instance_id for consistent coloring
    annotations = sorted(annotations, key=lambda x: (x.get('category_id', 0), x.get('instance_id', 0)))
    
    # Generate colors for instances
    instance_colors = get_distinct_colors(len(annotations))
    
    # Prepare image arrays
    img_array = np.array(img)
    
    # Determine subplot layout
    num_plots = sum([show_bbox, show_mask])
    if num_plots == 0:
        num_plots = 1
        show_bbox = True
    
    fig, axes = plt.subplots(1, num_plots, figsize=(10 * num_plots, 10))
    if num_plots == 1:
        axes = [axes]
    
    plot_idx = 0
    
    # Plot with bounding boxes
    if show_bbox:
        ax = axes[plot_idx]
        ax.imshow(img)
        ax.set_title(f"Bounding Boxes\n{image_info['file_name']}", fontsize=14, fontweight='bold')
        ax.axis('off')
        
        for i, ann in enumerate(annotations):
            bbox = ann.get('bbox')
            if bbox:
                color = instance_colors[i]
                rect = patches.Rectangle(
                    (bbox[0], bbox[1]), bbox[2], bbox[3],
                    linewidth=2.5, edgecolor=color, facecolor='none'
                )
                ax.add_patch(rect)
        
        plot_idx += 1
    
    # Plot with segmentation masks
    if show_mask:
        ax = axes[plot_idx]
        mask_overlay = img_array.copy().astype(float)
        
        for i, ann in enumerate(annotations):
            segmentation = ann.get('segmentation')
            if segmentation:
                mask = create_mask_from_segmentation(segmentation, img.size)
                if mask is not None:
                    color = instance_colors[i]
                    mask_color = np.array(to_rgba(color, alpha=0.6)[:3]) * 255
                    alpha = 0.6
                    for c in range(3):
                        mask_overlay[:, :, c] = np.where(
                            mask,
                            mask_overlay[:, :, c] * (1 - alpha) + mask_color[c] * alpha,
                            mask_overlay[:, :, c]
                        )
        
        ax.imshow(mask_overlay.astype(np.uint8))
        ax.set_title(f"Segmentation Masks\n{image_info['file_name']}", fontsize=14, fontweight='bold')
        ax.axis('off')
    
    # Create legend
    legend_entries = []
    for i, ann in enumerate(annotations):
        category_name = ann.get('category_name', categories.get(ann['category_id'], 'Unknown'))
        instance_id = ann.get('instance_id', '?')
        label = f"{category_name} (Inst:{instance_id})"
        legend_entries.append((instance_colors[i], label))
    
    # Add legend
    if legend_entries:
        handles = [patches.Patch(color=color, label=label) for color, label in legend_entries]
        fig.legend(handles=handles, loc='lower center', ncol=min(4, len(legend_entries)), 
                  fontsize=10, frameon=True, fancybox=True, shadow=True)
        plt.tight_layout(rect=[0, 0.08, 1, 1])
    else:
        plt.tight_layout()
    
    plt.show()
    
    # Print statistics
    print("="*70)
    print(f"📸 Image: {os.path.basename(img_path)}")
    print(f"📏 Size: {img.size[0]} × {img.size[1]} pixels")
    print(f"🆔 Image ID: {image_info['id']}")
    print(f"📊 Total Instances: {len(annotations)}")
    
    # Count by supercategory
    supercat_counts = defaultdict(int)
    for ann in annotations:
        cat_name = ann.get('category_name', categories.get(ann['category_id'], 'Unknown'))
        supercat = category_to_supercategory.get(cat_name, 'Unknown')
        supercat_counts[supercat] += 1
    
    print(f"\n🏷️  Instances by Supercategory:")
    for supercat, count in sorted(supercat_counts.items()):
        print(f"    • {supercat}: {count}")
    
    # Count instances with masks
    masks_count = sum(1 for ann in annotations if ann.get('segmentation'))
    print(f"\n🎭 Instances with Masks: {masks_count}/{len(annotations)}")
    print("="*70)


print("✅ Visualization function loaded successfully!")

✅ Visualization function loaded successfully!


In [11]:
# ===========================
# EXPLORATION FUNCTIONS
# ===========================

def explore_random(n=1, show_bbox=True, show_mask=True):
    """
    Display n random images from the dataset.
    
    Args:
        n: Number of random images to display
        show_bbox: Whether to show bounding boxes
        show_mask: Whether to show segmentation masks
    """
    sample_images = random.sample(data['images'], min(n, len(data['images'])))
    
    print("\n" + "🎲 " * 35)
    print(f"EXPLORING {len(sample_images)} RANDOM IMAGE(S)")
    print("🎲 " * 35 + "\n")
    
    for i, image_info in enumerate(sample_images, 1):
        print(f"\n{'#'*70}")
        print(f"# Random Image {i}/{len(sample_images)}")
        print(f"{'#'*70}\n")
        visualize_image(image_info, show_bbox=show_bbox, show_mask=show_mask)


def explore_by_supercategory(supercategory, n=1, show_bbox=True, show_mask=True):
    """
    Display n random images containing the specified supercategory.
    
    Args:
        supercategory: Supercategory name (e.g., 'Biped', 'Quadruped', 'Bird', etc.)
        n: Number of random images to display
        show_bbox: Whether to show bounding boxes
        show_mask: Whether to show segmentation masks
    """
    # Find images with this supercategory
    matching_images = [
        img for img in data['images']
        if supercategory in image_to_supercategories.get(img['id'], set())
    ]
    
    if not matching_images:
        print(f"❌ No images found with supercategory: {supercategory}")
        print(f"Available supercategories: {', '.join(supercategories)}")
        return
    
    sample_images = random.sample(matching_images, min(n, len(matching_images)))
    
    print("\n" + "🏷️  " * 35)
    print(f"EXPLORING {len(sample_images)} RANDOM IMAGE(S) WITH SUPERCATEGORY: {supercategory.upper()}")
    print(f"Found {len(matching_images)} total images with this supercategory")
    print("🏷️  " * 35 + "\n")
    
    for i, image_info in enumerate(sample_images, 1):
        print(f"\n{'#'*70}")
        print(f"# {supercategory} Image {i}/{len(sample_images)}")
        print(f"{'#'*70}\n")
        visualize_image(image_info, show_bbox=show_bbox, show_mask=show_mask)


def explore_by_id(image_id, show_bbox=True, show_mask=True):
    """
    Display a specific image by its ID.
    
    Args:
        image_id: Image ID to display
        show_bbox: Whether to show bounding boxes
        show_mask: Whether to show segmentation masks
    """
    image_info = next((img for img in data['images'] if img['id'] == image_id), None)
    
    if image_info is None:
        print(f"❌ Image with ID {image_id} not found")
        print(f"Available IDs: {sorted([img['id'] for img in data['images']])[:20]}...")
        return
    
    print("\n" + "🔍 " * 35)
    print(f"DISPLAYING SPECIFIC IMAGE (ID: {image_id})")
    print("🔍 " * 35 + "\n")
    
    visualize_image(image_info, show_bbox=show_bbox, show_mask=show_mask)


def explore_by_filename(filename, show_bbox=True, show_mask=True):
    """
    Display a specific image by its filename.
    
    Args:
        filename: Image filename (with or without extension)
        show_bbox: Whether to show bounding boxes
        show_mask: Whether to show segmentation masks
    """
    # Try exact match first
    image_info = next((img for img in data['images'] if img['file_name'] == filename), None)
    
    # Try without extension
    if image_info is None:
        image_info = next((img for img in data['images'] 
                          if img['file_name'].split('.')[0] == filename.split('.')[0]), None)
    
    if image_info is None:
        print(f"❌ Image with filename '{filename}' not found")
        print(f"Available filenames (first 10): {[img['file_name'] for img in data['images'][:10]]}")
        return
    
    print("\n" + "🔍 " * 35)
    print(f"DISPLAYING SPECIFIC IMAGE: {filename}")
    print("🔍 " * 35 + "\n")
    
    visualize_image(image_info, show_bbox=show_bbox, show_mask=show_mask)


def list_supercategories():
    """Display all available supercategories with image counts."""
    print("\n" + "="*70)
    print("📋 AVAILABLE SUPERCATEGORIES")
    print("="*70)
    
    for supercat in supercategories:
        # Count images with this supercategory
        image_count = sum(1 for img in data['images'] 
                         if supercat in image_to_supercategories.get(img['id'], set()))
        
        # Count total instances
        instance_count = sum(1 for ann in data['annotations']
                           if category_to_supercategory.get(
                               categories.get(ann['category_id'], 'Unknown'), 'Unknown'
                           ) == supercat)
        
        print(f"\n🏷️  {supercat}:")
        print(f"    • {image_count} images")
        print(f"    • {instance_count} instances")
        
        # List categories
        cat_ids = supercategory_to_categories[supercat]
        cat_names = [categories[cid] for cid in cat_ids if cid in categories]
        print(f"    • Parts: {', '.join(sorted(cat_names))}")
    
    print("\n" + "="*70)


print("✅ Exploration functions loaded successfully!")

✅ Exploration functions loaded successfully!


---
## 🎯 Quick Start Guide

### Available Functions:

1. **`list_supercategories()`** - View all available supercategories and statistics

2. **`explore_random(n=3)`** - View n random images from the dataset

3. **`explore_by_supercategory('Biped', n=2)`** - View random images containing specific supercategory
   - Available: Biped, Quadruped, Bird, Fish, Snake, Reptile, Car, Bicycle, Boat, Aeroplane, Bottle

4. **`explore_by_id(123)`** - View specific image by ID

5. **`explore_by_filename('n01440764_1108')`** - View specific image by filename

### Optional Parameters (all functions):
- `show_bbox=True/False` - Show/hide bounding boxes
- `show_mask=True/False` - Show/hide segmentation masks

---

In [12]:
# ===========================
# VIEW SUPERCATEGORIES
# ===========================

# List all available supercategories with statistics
list_supercategories()


📋 AVAILABLE SUPERCATEGORIES

🏷️  Aeroplane:
    • 15 images
    • 138 instances
    • Parts: Aeroplane Body, Aeroplane Engine, Aeroplane Head, Aeroplane Tail, Aeroplane Wing

🏷️  Bicycle:
    • 32 images
    • 146 instances
    • Parts: Bicycle Body, Bicycle Head, Bicycle Seat, Bicycle Tire

🏷️  Biped:
    • 60 images
    • 309 instances
    • Parts: Biped Arm, Biped Body, Biped Head, Biped Leg, Biped Tail

🏷️  Bird:
    • 60 images
    • 309 instances
    • Parts: Bird Body, Bird Foot, Bird Head, Bird Tail, Bird Wing

🏷️  Boat:
    • 28 images
    • 83 instances
    • Parts: Boat Body, Boat Sail

🏷️  Bottle:
    • 24 images
    • 44 instances
    • Parts: Bottle Body, Bottle Mouth

🏷️  Car:
    • 60 images
    • 268 instances
    • Parts: Car Body, Car Side Mirror, Car Tire

🏷️  Fish:
    • 60 images
    • 363 instances
    • Parts: Fish Body, Fish Fin, Fish Head, Fish Tail

🏷️  Quadruped:
    • 60 images
    • 273 instances
    • Parts: Quadruped Body, Quadruped Foot, Quadruped Head

---
## 🎲 Example 1: Explore Random Images

In [13]:
# View 3 random images from the entire dataset
explore_random(n=3)


🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 
EXPLORING 3 RANDOM IMAGE(S)
🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 🎲 


######################################################################
# Random Image 1/3
######################################################################

❌ Could not load image: val/n02025239_3674.jpeg

######################################################################
# Random Image 2/3
######################################################################

❌ Could not load image: val/n04557648_731.jpeg

######################################################################
# Random Image 3/3
######################################################################

❌ Could not load image: val/n01748264_6780.jpeg


---
## 🏷️ Example 2: Explore by Supercategory

In [ ]:
# View random images containing Biped parts
explore_by_supercategory('Biped', n=2)

In [ ]:
# View random images containing Quadruped parts
explore_by_supercategory('Quadruped', n=2)

In [ ]:
# View random images containing Bird parts
explore_by_supercategory('Bird', n=2)

---
## 🔍 Example 3: Explore Specific Images

In [ ]:
# View specific image by ID
# Change the ID to explore different images
explore_by_id(1)

In [ ]:
# View specific image by filename
# Change the filename to explore different images
explore_by_filename('n01440764_1108')

---
## 🎨 Example 4: Custom Visualization Options

In [ ]:
# Show only bounding boxes (no masks)
explore_random(n=1, show_bbox=True, show_mask=False)

In [ ]:
# Show only segmentation masks (no bounding boxes)
explore_random(n=1, show_bbox=False, show_mask=True)

---
## 💡 Tips

- **Explore different supercategories**: Try 'Car', 'Bicycle', 'Aeroplane', 'Boat', 'Bottle', 'Fish', 'Snake', 'Reptile'
- **Find image IDs**: Check the output after loading the dataset or use `list_supercategories()`
- **Multiple images**: Increase `n` parameter to view more images at once
- **Performance**: Viewing many images at once (n > 10) may slow down the notebook
- **Switch splits**: Go back to the configuration cell and change `DATASET_SPLIT` to view train/test/val

---

---
## 🚀 Interactive Exploration Cell

**Run the cell below and modify the function call to explore the dataset!**

In [ ]:
# ===========================
# YOUR EXPLORATION CELL
# ===========================
# Modify this cell to explore the dataset!

# Example: Uncomment one of the lines below or write your own

# explore_random(n=5)
# explore_by_supercategory('Biped', n=3)
# explore_by_supercategory('Car', n=2)
# explore_by_id(1)
# explore_by_filename('n01440764_1108')

# Your code here:
explore_random(n=1)